In [1]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

# Lectura de Datos

In [2]:
items_df = pd.read_csv('Data/items_titles.csv'
                       , encoding = 'UTF-8'
                      )

In [3]:
# Generar columna de texto procesado
items_df['processed_titles'] = items_df['ITE_ITEM_TITLE'].str.lower().str.replace('[^\w\s]', '', regex=True)

In [4]:
items_df.head(5)

,ITE_ITEM_TITLE,processed_titles
0,Tênis Ascension Posh Masculino - Preto E Verme...,tênis ascension posh masculino preto e vermelho
1,Tenis Para Caminhada Super Levinho Spider Corr...,tenis para caminhada super levinho spider corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,tênis feminino le parc hocks blackice original...
3,Tênis Olympikus Esportivo Academia Nova Tendên...,tênis olympikus esportivo academia nova tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,inteligente led bicicleta tauda luz usb bicicl...


## Word Embeddings

In [5]:
# Pasar elemntos de la oración a listas
items_df['processed_titles_list'] = items_df['processed_titles'].str.split()
items_list = items_df['processed_titles_list'].to_list()

In [6]:
# Generacion de modelo
modelo = Word2Vec(items_df['processed_titles'].tolist()
                  , vector_size=100
                  , window=7
                  , min_count=3
                  , workers=12
                 )
modelo.save("word2vec.modelo_00")

In [7]:
# Mapear vector para cada item

items_df['vector'] = items_df['processed_titles'].apply(lambda x: np.mean([modelo.wv[word] for word in x if word in modelo.wv], axis=0)) 

In [8]:
vectors = np.stack(items_df['vector'].values)

# Calcular la matriz de similitud del coseno
cosine_sim_matrix = cosine_similarity(vectors)

# Opcional: Convertir la matriz de similitud del coseno a un DataFrame para una mejor visualización
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=items_df[items_df.columns[0]], columns=items_df[items_df.columns[0]])


In [9]:
cosine_sim_df['Tenis Botinha Com Pelo Via Marte Original Lançamento'].sort_values(ascending=False)

ITE_ITEM_TITLE
Tenis Botinha Com Pelo Via Marte Original Lançamento           1.000000
Botinha Com Pelo Tenis Via Marte Outono Inverno Lançamento     0.992257
Tenis Moleca Feminino Camurça Sapato Original + Brinde         0.989448
Tenis Masculino Polo Open Original Barato Envio Imediato       0.989260
Tenis Masculino Barato Academia Esportivo Leve Lançamento      0.988686
                                                                 ...   
Tenis Kidy Colors  009-0857-0312 009-0857-0312                -0.234610
Pegada 15505 38/43                                            -0.236634
Tenis Klin Jump -pink 244.022000-000334                       -0.236957
Slip On Klin New Sport  199.075000-000143 199.075000-000143   -0.254822
Tenis Klin Freestyle  260.043000-000001 260.043000-000001     -0.254945
Name: Tenis Botinha Com Pelo Via Marte Original Lançamento, Length: 30000, dtype: float32

## TF-IDF